In [1]:
import torchvision
from PIL import Image
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torchvision.transforms.functional import to_tensor
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import os
from google.colab import files
import torchvision.models as models
from torchvision.transforms import Resize
import torch

In [2]:
#Upload the dataset.zip or the image
uploaded = files.upload()

Saving original_water.jpg to original_water.jpg


In [ ]:
!pwd

/content


In [ ]:
!ls

my_data  original_water.jpg  sample_data


In [3]:
#d_driven = True if data driven approach want to be used
d_driven = False

In [4]:
#unzip the dataset
if(d_driven):
  !unzip dotted.zip -d /content

In [5]:
#Create a folder for the results
!mkdir -p /content/my_data

In [6]:
#configurable parameters
lr = 0.0002
batch_size = 8
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 100

In [7]:
if(d_driven):
    textureName = "data_driven_dotted"

In [8]:
#Open the original image and convert it in np.array and define the number of random crops
if(not d_driven):
    imagePath = "original_water.jpg"
    image = Image.open(imagePath).convert('RGB')
    image = np.array(image)
    textureName = "water-texture"
    size = 50

In [9]:
#The dataset class. Based on d_driven decides to proced with data-free or data-driven approach
class Textures(Dataset):
    def __init__(self, principal_image=None, size=0, directory=""):
        super(Textures, self).__init__()
        self.principal_image = principal_image
        self.data = size

        self.trans = transforms.Compose([
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
            transforms.Resize((1024, 1024)),
            transforms.RandomCrop((512, 512))
        ])

        if(d_driven):
            self.image_paths = [os.path.join(directory, filename) for filename in os.listdir(directory) if filename.endswith('.jpg') ]
        else:
            self.principal_image = principal_image
            self.data = size
            self.principal_image = to_tensor(self.principal_image)

    def __len__(self):
        if(d_driven):
            return len(self.image_paths)
        return self.data

    def __getitem__(self, index):

        if(d_driven):
            image_path = self.image_paths[index]
            image = Image.open(image_path).convert('RGB')
            image = np.array(image)
            image = to_tensor(image)
            image = self.trans(image)
            return image

        image = self.trans(self.principal_image)

        return image

In [10]:
#Defines the Discriminator class
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, 2, 1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1024, 4, 1, 0),
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(1024, 1, 4, 1, 0),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)


In [11]:
#Defines the Generator class
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()


        self.first = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=1, padding=3, padding_mode="reflect"),
            nn.ReLU(inplace=True)
        )

        self.downSampling = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.InstanceNorm2d(256),
            nn.ReLU(inplace=True)
        )

        self.residualBlocks = nn.Sequential(*[self._make_residual_block(256) for _ in range(9)])

        self.upSampling = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(256),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True)
        )

        self.last = nn.Conv2d(64, 3, kernel_size=7, stride=1, padding=3, padding_mode="reflect")



    def _make_residual_block(self, input):
        block = nn.Sequential(
            nn.Conv2d(input, input, kernel_size=3, padding=1),
            nn.InstanceNorm2d(input),
            nn.ReLU(inplace=True),
            nn.Conv2d(input, input, kernel_size=3, padding=1),
            nn.InstanceNorm2d(input),
            nn.ReLU(inplace=True)
        )
        return block

    def forward(self, x):
        x = self.first(x)
        x = self.downSampling(x)
        skip = x
        x = self.residualBlocks(x)
        x = torch.cat([x, skip], dim=1)
        x = self.upSampling(x)
        x = self.last(x)
        return torch.tanh(x)


In [12]:
#Defines the Loss function
class PerceptualLoss(nn.Module):
    def __init__(self):
        super(PerceptualLoss, self).__init__()
        self.vgg = models.vgg16(pretrained=True).features[:16].eval()
        for param in self.vgg.parameters():
            param.requires_grad = False

    def forward(self, generated, target):
        generated_features = self.vgg(generated)
        target_features = self.vgg(target)
        loss = nn.functional.mse_loss(generated_features, target_features)
        return loss

In [13]:
#initialize the Loss
criterion_GAN = nn.BCELoss()
criterion_perceptual = PerceptualLoss().to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:05<00:00, 93.4MB/s]


In [14]:
#The main function used for the training
def train(discriminator, generator, dataloader, optim_D, optim_G):

    for epoch in range(epochs):
        loop = tqdm(dataloader, leave=True)

        for i, real_images in enumerate(loop):

            original_shapes = real_images.shape
            new_shape = (original_shapes[0], original_shapes[1], original_shapes[2] // 2, original_shapes[3] // 2)

            real_images = real_images.to(device)

            noise = torch.normal(0, 1, size=new_shape).to(device)
            fake_images = generator(noise)

            # Allenamento del discriminatore
            optim_D.zero_grad()
            real_preds = discriminator(real_images).squeeze()
            fake_preds = discriminator(fake_images.detach()).squeeze()
            loss_D_real = criterion_GAN(real_preds, torch.ones_like(real_preds))
            loss_D_fake = criterion_GAN(fake_preds, torch.zeros_like(fake_preds))
            loss_D = (loss_D_real + loss_D_fake) / 2
            loss_D.backward()
            optim_D.step()

            # Allenamento del generatore
            optim_G.zero_grad()
            fake_preds = discriminator(fake_images).squeeze()
            loss_G_GAN = criterion_GAN(fake_preds, torch.ones_like(fake_preds))
            loss_G_perceptual = criterion_perceptual(fake_images, real_images)
            loss_G = loss_G_GAN + loss_G_perceptual
            loss_G.backward()
            optim_G.step()

            loop.set_postfix(Disc_Loss=loss_D.item(), Gen_Loss=loss_G.item(), Epoch=epoch)

    # Salvataggio del modello del Generatore
    torch.save(generator.state_dict(), f'generator_{textureName}.pth')
    files.download(f'generator_{textureName}.pth')

In [15]:
#initialize the Discriminator, the Generator, Adam optimizers and the DataLoader
discriminator = Discriminator().to("cuda")
generator = Generator().to("cuda")

optim_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
optim_G = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))

if(d_driven):
    dataset = Textures(directory = "dotted")
else:
    dataset = Textures(principal_image=image, size = size)

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [16]:
#call the training function
train(discriminator, generator, dataloader, optim_D, optim_G)

  0%|          | 0/7 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
100%|██████████| 7/7 [00:19<00:00,  2.75s/it, Disc_Loss=0.355, Epoch=99, Gen_Loss=5.03]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
#In the case of data-free approach open the image and convert it to a tensor
if(not d_driven):
    image = Image.open(imagePath).convert('RGB')
    image = np.array(image)
    image = to_tensor(image)

In [18]:
#Define a transformation
trans = transforms.Compose([
    transforms.RandomCrop((512, 512)),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [19]:
#loading the Generator and putting it on eval mode
gen = Generator().to(device)
gen.load_state_dict(torch.load(f'generator_{textureName}.pth'))
gen.eval()

Generator(
  (first): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), padding_mode=reflect)
    (1): ReLU(inplace=True)
  )
  (downSampling): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (2): ReLU(inplace=True)
    (3): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (5): ReLU(inplace=True)
  )
  (residualBlocks): Sequential(
    (0): Sequential(
      (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (2): ReLU(inplace=True)
      (3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=Fals

In [20]:
#In the case of data-free approach create 10 random crop images from the original image.
if(not d_driven):
    images = [trans(image).unsqueeze(0).to(device) for i in range(10)]
    resize_transform = Resize((512, 512))

In [21]:
#function used to save and download the result both in the data-free and data-driven approach
def save_and_download(output, i):
    output = output * 0.5 + 0.5
    save_image(output, f"my_data/{textureName}_{i}.png")
    files.download(f"my_data/{textureName}_{i}.png")

In [24]:
#generate images using the newly trained model

image = images[0]

for i in range(7):
    if(d_driven):
        noise = torch.normal(0, 1, size=(8, 3, 256, 256)).to(device)
        output = gen(noise)
    else:
        noise = torch.normal(0, 1, size=image.shape).to(device)
        output = gen(noise)
        output = resize_transform(output)

    save_and_download(output, i)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>